In [ ]:
print('running')
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd
import numpy as np
import plotly.graph_objects as go
pd.set_option('float_format', '{:f}'.format)
from datetime import datetime
import plotly.figure_factory as ff
import dash_table
from statistics import mean


Runs=['run_1','run_2','run_3','run_4','run_5','run_6','run_7','run_8','run_9','run_10','run_11','run_12','run_13','run_14','run_15','run_16','run_17','run_18']
%store -r Dataframes
%store -r Dataframes_Jis
%store -r Dataframes_Worker
%store -r Dataframes_Station
%store -r Dataframes_instation
%store -r utilization_list 
%store -r utilization_list_worker

#df=Dataframes_Worker[0]
dict_worker={'run','percentage working','percentage idle','percentage rotation','average rotations (1/h)'}
df_t = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/solar.csv')
#dict_worker['run'][0]=1

name_list=list()
average_list_working=list()
average_list_idle=list()
average_list_rotation=list()
average_list_rotations=list()
for i in range(len(Dataframes_Worker)):
    name_list.append('run_'+str(i+1))
    average_list_working.append(Dataframes_Worker[i]['percentage working'].mean())
    average_list_idle.append(Dataframes_Worker[i]['percentage idle'].mean())
    average_list_rotation.append(Dataframes_Worker[i]['percentage rotation'].mean())
    average_list_rotations.append(Dataframes_Worker[i]['average rotations (1/h)'].mean())
    
data={'run':name_list,
      'percentage working':average_list_working,
      'percentage idle':average_list_idle,
      'percentage rotation':average_list_rotation,
      'average rotations (1/h)':average_list_rotations}

df = pd.DataFrame(data)

df_dict=df.to_dict()

In [ ]:
#Function for the calculating the simult. Jis wagon
def simul(jis_df):
    start_time=int(jis_df['Jis-wagon opening'][0])
    end_time=int(jis_df['Jis-wagon completion'][len(jis_df)-1])
    simul=np.zeros(end_time-start_time)
    for i in range(len(jis_df)):
        start_jis=int(jis_df['Jis-wagon opening'][i]-start_time)
        end_jis=int(jis_df['Jis-wagon completion'][i]-start_time)
        for j in range(end_jis-start_jis):
            simul[start_jis+j]+=1
                
    simul_df = pd.DataFrame(simul,columns=['simul'])
    
    return simul_df

In [ ]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

#auth = dash_auth.BasicAuth(
 #   app,
#    VALID_USERNAME_PASSWORD_PAIRS
#)

all_comparison = {
    'product swirl': ['p-rate per worker (1/(min/worker))', 'p-rate hour (1/min)', 'p-rate hour average 10 (1/min)','p-rate (1/min)'],
    'Jis-wagon': ['Jis-wagon duration (min) - hist','Jis-wagon duration (min)','Jis-wagon development','time intervall (min) - hist','time intervall (min)','simultaneous Jis-wagon'],
    'utilization worker': ['worker percentage working', 'worker utilization'],
    'utilization station':['station percentage active','station percentage active working', 'station utilization']
}



app.layout = html.Div(
    children=[
        html.Div([

        html.Div([
            dcc.Dropdown(
                id='run',
                options=[
                        {'label': 'run_1', 'value': 'run_1'},
                        {'label': 'run_2', 'value': 'run_2'},
                        {'label': 'run_3', 'value': 'run_3'},
                        {'label': 'run_4', 'value': 'run_4'},
                        {'label': 'run_5', 'value': 'run_5'},
                        {'label': 'run_6', 'value': 'run_6'},
                        {'label': 'run_7', 'value': 'run_7'},
                        {'label': 'run_8', 'value': 'run_8'},
                        {'label': 'run_9', 'value': 'run_9'},
                        {'label': 'run_10', 'value': 'run_10'},
                        {'label': 'run_11', 'value': 'run_11'},
                        {'label': 'run_12', 'value': 'run_12'},
                        {'label': 'run_13', 'value': 'run_13'},
                        {'label': 'run_14', 'value': 'run_14'},
                        {'label': 'run_15', 'value': 'run_15'},
                        {'label': 'run_16', 'value': 'run_16'},
                        {'label': 'run_17', 'value': 'run_17'},
                        {'label': 'run_18', 'value': 'run_18'}
        ],
            value='run_1',
            multi=True
    ),
            dcc.Dropdown(
                id='pre-comparison',
                options=[{'label': k, 'value': k} for k in all_comparison.keys()],
                value='product swirl',
                multi=False),
            
            dcc.Dropdown(
                id='comparison',
                value='p-rate per worker (1/(min/worker))'
            )
            
        ],
        style={'width': '48%', 'display': 'inline-block'}),
    ]),

    dcc.Graph(id='graph'
             ),
    
    dash_table.DataTable(
    id='data-table',
 #   columns=[{"name": i, "id": i} 
  #               for i in df.columns],
   # data=df.to_dict('records'),
)
])

@app.callback(
    Output('run', 'multi'),
    Input('comparison','value'))
def set_multi(pre_comparison):
    if pre_comparison=='Jis-wagon development' or pre_comparison=='station utilization' or pre_comparison=='worker utilization':
        return False
    else:
        return True
@app.callback(
    Output('comparison','options'),
    Input('pre-comparison','value'))
def set_comparison(pre_comparison):
    print(all_comparison[pre_comparison])
    return [{'label': i, 'value': i} for i in all_comparison[pre_comparison]]




@app.callback(
    Output('graph','figure'),
    Input('run','value'),
    Input('comparison','value'),
    Input('pre-comparison','value'))
def update_graph(run,comp,pre_comp):
    if pre_comp=='product swirl':
        df_list=list()
        label_list=list()
        print(type(run))
        print(run)
        if type(run)==str:
            for i in range(len(Runs)):
                if run==Runs[i]:
                    df=Dataframes[i]
                    line_name=Runs[i]
            print(line_name)
            fig=go.Figure()
            fig.add_trace(go.Scatter(x=df['finish index'],y=df[comp],mode='lines',name=line_name))
            return fig

        else:
            print('list')
            print(len(run))
            for i in range(len(run)):
                for b in range(len(Runs)):
                    if run[i]==Runs[b]:
                        df_list.append(Dataframes[b])
                        label_list.append(Runs[b])


            print(len(df_list))
            fig = go.Figure()
            for i in range(len(label_list)):
                fig.add_trace(go.Scatter(x=df_list[i]['finish index'], y=df_list[i][comp],mode='lines',name=label_list[i]))

            return fig
        
    if comp=='Jis-wagon duration (min) - hist':
        df_list=list()
        value_list=list()
        if type(run)==str:
            fig=px.histogram(x=Dataframes_Jis[0]['Jis-wagon duration (min)'])

            
        
        else:
            for i in range(len(run)):
                for b in range(len(Runs)):
                    if run[i]==Runs[b]:
                        df_list.extend(Dataframes_Jis[b]['Jis-wagon duration (min)'].values.tolist())
                        value_list.extend([Runs[b]]*len(Dataframes_Jis[i]['Jis-wagon duration (min)']))

            df=pd.DataFrame(dict(
                series=value_list,
                data=df_list))
        
        
            fig=px.histogram(df,x='data',color='series',barmode='overlay')
            
            
    if comp=='Jis-wagon development':
        for i in range(len(Runs)):
            if run==Runs[i]:
                jis_df=Dataframes_Jis[i]
        fig = go.Figure(
            data=[
                go.Bar(
                name='Jis-wagon duration (min)',
                y=jis_df['Jis-wagon nr'],
                x=jis_df['Jis-wagon duration (min)'],
                offsetgroup=0,
                base=jis_df["sim time (min)"],
                orientation='h'
                ),
                go.Bar(name='Time Intervall',
                y=jis_df['Jis-wagon nr'],
                x=jis_df['time intervall (min)'],
                base=(jis_df['Jis-wagon duration (min)']+jis_df['sim time (min)']),
                offsetgroup=1,
                orientation='h'
                )
            ],
            layout=go.Layout(
                yaxis_title='Time in min',
                xaxis_title='JIS-wagon'))
        fig['layout']['yaxis']['autorange'] = "reversed"
        
    
    if comp=='time intervall (min) - hist':
        df_list=list()
        value_list=list()
        if type(run)==str:
            fig=px.histogram(x=Dataframes_Jis[0]['time intervall (min)'])

            
        
        else:
            for i in range(len(run)):
                for b in range(len(Runs)):
                    if run[i]==Runs[b]:
                        df_list.extend(Dataframes_Jis[b]['time intervall (min)'].values.tolist())
                        value_list.extend([Runs[b]]*len(Dataframes_Jis[i]['time intervall (min)']))

            df=pd.DataFrame(dict(
                series=value_list,
                data=df_list))
        
        
            fig=px.histogram(df,x='data',color='series',barmode='overlay')
            
    if comp=='time intervall (min)':
        df_list=list()
        label_list=list()
        if type(run)==str:
            for i in range(len(Runs)):
                if run==Runs[i]:
                    df=Dataframes_Jis[i]
                    line_name=Runs[i]
            print(line_name)
            fig=go.Figure()
            fig.add_trace(go.Bar(x=df_list[i]['finish index'],y=df['time intervall (min)'],name=line_name))
            fig.update_layout(barmode='group')
            
            return fig

        else:
            print('list')
            print(len(run))
            for i in range(len(run)):
                for b in range(len(Runs)):
                    if run[i]==Runs[b]:
                        df_list.append(Dataframes_Jis[b])
                        label_list.append(Runs[b])


            print(len(df_list))
            fig = go.Figure()
            for i in range(len(label_list)):
                fig.add_trace(go.Bar(x=df_list[i]['finish index'], y=df_list[i]['time intervall (min)'],name=label_list[i]))
            fig.update_layout(barmode='group')

            return fig
        
    if comp=='Jis-wagon duration (min)':
        df_list=list()
        label_list=list()
        if type(run)==str:
            for i in range(len(Runs)):
                if run==Runs[i]:
                    df=Dataframes_Jis[i]
                    line_name=Runs[i]
            print(line_name)
            fig=go.Figure()
            fig.add_trace(go.Bar(x=df_list[i]['finish index'],y=df['Jis-wagon duration (min)'],name=line_name))
            fig.update_layout(barmode='group')
            
            return fig

        else:
            print('list')
            print(len(run))
            for i in range(len(run)):
                for b in range(len(Runs)):
                    if run[i]==Runs[b]:
                        df_list.append(Dataframes_Jis[b])
                        label_list.append(Runs[b])


            print(len(df_list))
            fig = go.Figure()
            for i in range(len(label_list)):
                fig.add_trace(go.Bar(x=df_list[i]['finish index'], y=df_list[i]['Jis-wagon duration (min)'],name=label_list[i]))
            fig.update_layout(barmode='group')
            
    
    if comp=='simultaneous Jis-wagon':
        df_list=list()
        label_list=list()
        if type(run)==str:
            for i in range(len(Runs)):
                if run==Runs[i]:
                    df=simul(Dataframes_Jis[i])
                    line_name=Runs[i]
                    

                    
            fig=go.Figure()
            fig.add_trace(go.Scatter(y=df['simul'],mode='lines',name=line_name))
            return fig

        else:
            for i in range(len(run)):
                for b in range(len(Runs)):
                    if run[i]==Runs[b]:
                        df_list.append(simul(Dataframes_Jis[b]))
                        label_list.append(Runs[b])


            print(len(df_list))
            fig = go.Figure()
            for i in range(len(label_list)):
                fig.add_trace(go.Scatter(y=df_list[i]['simul'],mode='lines',name=label_list[i]))

            return fig
        
        
    if comp=='station percentage active':
        df_list=list()
        label_list=list()
        if type(run)==str:
            for i in range(len(Runs)):
                if run==Runs[i]:
                    df=Dataframes_Station[i]
                    line_name=Runs[i]
            fig=go.Figure()
            fig.add_trace(go.Bar(x=df['station name'],y=df['percentage active'],name=line_name))
            fig.update_layout(barmode='group')
            
            return fig

        else:
            for i in range(len(run)):
                for b in range(len(Runs)):
                    if run[i]==Runs[b]:
                        df_list.append(Dataframes_Station[b])
                        label_list.append(Runs[b])


            print(len(df_list))
            fig = go.Figure()
            for i in range(len(label_list)):
                fig.add_trace(go.Bar(x=df_list[i]['station name'], y=df_list[i]['percentage active'],name=label_list[i]))
            fig.update_layout(barmode='group')

            return fig
        
        
    if comp=='station percentage active working':
        df_list=list()
        label_list=list()
        if type(run)==str:
            for i in range(len(Runs)):
                if run==Runs[i]:
                    df=Dataframes_Station[i]
                    line_name=Runs[i]
            fig=go.Figure()
            fig.add_trace(go.Bar(x=df['station name'],y=df['percentage working active'],name=line_name))
            fig.update_layout(barmode='group')
            
            return fig

        else:
            for i in range(len(run)):
                for b in range(len(Runs)):
                    if run[i]==Runs[b]:
                        df_list.append(Dataframes_Station[b])
                        label_list.append(Runs[b])


            print(len(df_list))
            fig = go.Figure()
            for i in range(len(label_list)):
                fig.add_trace(go.Bar(x=df_list[i]['station name'], y=df_list[i]['percentage working active'],name=label_list[i]))
            fig.update_layout(barmode='group')

            return fig
        
    if comp=='worker percentage working':
        df_list=list()
        label_list=list()
        if type(run)==str:
            for i in range(len(Runs)):
                if run==Runs[i]:
                    df=Dataframes_Worker[i]
                    line_name=Runs[i]
            fig=go.Figure()
            fig.add_trace(go.Bar(x=df['worker name'],y=df['percentage working'],name=line_name))
            fig.update_layout(barmode='group')
            
            return fig

        else:
            for i in range(len(run)):
                for b in range(len(Runs)):
                    if run[i]==Runs[b]:
                        df_list.append(Dataframes_Worker[b])
                        label_list.append(Runs[b])


            print(len(df_list))
            fig = go.Figure()
            for i in range(len(label_list)):
                fig.add_trace(go.Bar(x=df_list[i]['worker name'], y=df_list[i]['percentage working'],name=label_list[i]))
            fig.update_layout(barmode='group')

            return fig
        
    
    if comp=='station utilization':
        for i in range(len(Runs)):
            if run==Runs[i]:
                df=pd.DataFrame(utilization_list[i])
                
        fig = ff.create_gantt(df, index_col='Resource', show_colorbar=True,
                      group_tasks=True)
        
        
        
        
    if comp=='worker utilization':
        for i in range(len(Runs)):
            if run==Runs[i]:
                df=pd.DataFrame(utilization_list_worker[i])
                
        fig = ff.create_gantt(df, index_col='Resource', show_colorbar=True,
                      group_tasks=True)
        
        
    return fig  


@app.callback(
    [Output('data-table','columns'),
    Output('data-table','data')],
    [Input('run','value'),
    Input('pre-comparison','value')])
def update_data_table(run,pre_comp):
    if type(run)==str:
        run_list=list()
        run_list.append(run)
        run=run_list
    if pre_comp=='utilization worker':
        name_list=list()
        average_list_working=list()
        average_list_idle=list()
        average_list_rotation=list()
        average_list_rotations=list()
        print('utilization worker')
        for i in range(len(run)):
                for b in range(len(Runs)):
                    if run[i]==Runs[b]:
                        name_list.append(Runs[b])
                        average_list_working.append(Dataframes_Worker[b]['percentage working'].mean())
                        average_list_idle.append(Dataframes_Worker[b]['percentage idle'].mean())
                        average_list_rotation.append(Dataframes_Worker[b]['percentage rotation'].mean())
                        average_list_rotations.append(Dataframes_Worker[b]['average rotations (1/h)'].mean())
    
    
        data={'run':name_list,
          'percentage working':average_list_working,
          'percentage idle':average_list_idle,
          'percentage rotation':average_list_rotation,
          'average rotations (1/h)':average_list_rotations}
    


    
    if pre_comp=='product swirl':
        name_list=list()
        average_list_prate=list()
        average_list_pratehour=list()
        average_list_pratehour_init=list()
        average_list_prateworker=list()
        for i in range(len(run)):
                for b in range(len(Runs)):
                    if run[i]==Runs[b]:
                        name_list.append(Runs[b])
                        average_list_prate.append(Dataframes[b]['p-rate (1/min)'].mean())
                        average_list_pratehour.append(Dataframes[b]['p-rate hour (1/min)'].mean())
                        average_list_pratehour_init.append(Dataframes[b]['p-rate hour (1/min)'][200:-1].mean())
                        average_list_prateworker.append(Dataframes[b]['p-rate per worker (1/(min/worker))'].mean())
    
    
        data={'run':name_list,
          'p-rate':average_list_prate,
          'p-rate hour':average_list_pratehour,
          'p-rate hour after 200 jobs':average_list_pratehour_init,
          'p-rate per worker': average_list_prateworker}
        
        
    if pre_comp=='Jis-wagon':
        name_list=list()
        average_list_duration=list()
        average_list_timeintervall=list()
        average_list_simul=list()
        for i in range(len(run)):
                for b in range(len(Runs)):
                    if run[i]==Runs[b]:
                        name_list.append(Runs[b])
                        average_list_duration.append(Dataframes_Jis[b]['Jis-wagon duration (min)'].mean())
                        average_list_timeintervall.append(Dataframes_Jis[b]['time intervall (min)'].mean())
                        average_list_simul.append(simul(Dataframes_Jis[b]).mean())
    
    
        data={'run':name_list,
          'Jis-wagon duration (min)':average_list_duration,
          'time intervall (min)':average_list_timeintervall,
          'simultaneous Jis-wagon':average_list_simul}
        
        
    if pre_comp=='utilization station':
        name_list=list()
        average_list_active=list()
        average_list_working=list()
        for i in range(len(run)):
                for b in range(len(Runs)):
                    if run[i]==Runs[b]:
                        name_list.append(Runs[b])
                        average_list_active.append(Dataframes_Station[b]['percentage active'].mean())
                        average_list_working.append(Dataframes_Station[b]['percentage working active'].mean())
    
    
        data={'run':name_list,
          'percentage active':average_list_active,
          'percentage working active':average_list_working}
    

    df = pd.DataFrame(data)
        
    columns=[{"name": i, "id": i} 
                 for i in df.columns]
    
    data=df.to_dict('records')
    
    return columns,data
        
        
        
        
        
        
        


   


            
        
    
    
    


if __name__ == '__main__':
    app.run_server(debug=False)